<a href="https://colab.research.google.com/github/ursulacotrina/Project01/blob/main/scrapping_parlamento_periodico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 01 Install dependencies

In [54]:
!pip install --upgrade -q pygsheets

In [55]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
!pip install Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [57]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 http://deb.debian.org/debian buster InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:3 http://deb.debian.org/debian buster-updates InRelease
Hit:4 http://deb.debian.org/debian-security buster/updates InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu foca

In [ ]:
# Permite ejecutar comandos de shell directamente en tu entorno de Colab
%%shell

# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium

# Install xvfb
apt install -y xvfb

# Install Selenium-Profiles
pip uninstall -y selenium_profiles
pip install --no-cache-dir selenium_profiles>=2.2.6

# pip install https://github.com/kaliiiiiiiiii/Selenium-Profiles/archive/refs/heads/dev.zip # dev-branch

# install python packages
pip install google-colab-shell
pip install webdriver-manager
pip install Pyvirtualdisplay

Executing: /tmp/apt-key-gpghome.F1dqGiJENW/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.5kEQIEc92j/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.MQyKUOx0Lj/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

## 02 Install Packages

In [ ]:
import google.auth
from google.colab import auth
auth.authenticate_user()
import pygsheets
credentials, _ = google.auth.default()
gc = pygsheets.client.Client(credentials)

In [ ]:
import pandas as pd

from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium import webdriver
from selenium.webdriver.common.by import By

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
import warnings
import datetime
from datetime import datetime
import time

import Levenshtein  as lv
from itertools import combinations, permutations
import numpy as np

## 03 Functions

In [ ]:
def open_gsheets(link, sheet_index):
  sh1 = gc.open_by_url(link)
  wks = sh1[sheet_index]
  data_get = pd.DataFrame(wks.get_all_records())
  return data_get

In [ ]:
def combinations_string(phrases):
  p = phrases.split()
  return [" ".join(a) for a in permutations(p, len(p))]

In [ ]:
def match_phrases_target(list_1, list_2, nn):
  '''
  match_phrases_target

  Compare all word combinations of phrases versus a string target by Levenshtein algorithm, result is data frame with
  list of phrases, list of NN firsts string target more likely to match list of phrases, combination winner and ratio of match
  calculated by Levenshtein
  '''
  r  =pd.DataFrame(columns=['index', 'list_1_phrases', 'list_2_ccpp', 'combination', 'ratio'])
  nn = nn
  for i in list_1:
    #print('frase: ',i)
    k = []
    for l in list_2:
      w=[[],[]]
      for j in combinations_string(i):
        aa = lv.ratio(j, l)
        w[0].append(j)
        w[1].append(aa)
        #print('ccpp: '+l, 'combinación: '+j, aa)
      a_max = max(w[1])
      a_max_arg = w[0][w[1].index(max(w[1]))]
      k.append([i, l, a_max_arg,a_max])
    #print(k)
    dd = pd.DataFrame(k, columns=['list_1_phrases', 'list_2_ccpp', 'combination', 'ratio']).sort_values(
      by=['list_1_phrases','ratio'], ascending=False).reset_index()[0:nn]
    r = r.append(dd, ignore_index=True)
  return r

## 04 Import data

### 04 01 scrapkeys

In [ ]:
link = 'https://docs.google.com/spreadsheets/d/1FQeZHvmFS5SAIMgGkpP7KNW7CxWu2cP95SkjZqTCh34/edit#gid=0'
sheet_index = 0
variables1 = open_gsheets(link, sheet_index)
variables1

### 04 02 Acciones y evaluación

In [ ]:
link = 'https://docs.google.com/spreadsheets/d/1unHBIVHSB_oqrtDQb7bNunzPiC_VT_GDXjzRFyTTyXw/edit#gid=0'
sheet_index = 0
variables2 = open_gsheets(link, sheet_index)
variables2

In [ ]:
numbers = variables2['Numero'].tolist()
numbers

### 04 03 Datos congresistas

In [ ]:
link = 'https://docs.google.com/spreadsheets/d/1BAbFMVPSNuHdj5MaUZqLXgLYN8XA27ERrRbvovVrEHY/edit?usp=drive_web&ouid=117333138354521315740'

In [ ]:
sheet_index = 2
datoscongresistas = open_gsheets(link, sheet_index)
datoscongresistas.head(4)

## 05 Scrapping

In [ ]:
var = variables1['var'][variables1['featurestable']=='features'].tolist()
path = variables1['path'][variables1['featurestable']=='features']
path14 = variables1['path'][13]

estadoscrap = []
autorlink = []
fecha = []
enlace = []
vars=[]
vars = var + ['estadoscrap', 'autorlink', 'fecha', 'enlace']

# Convierte a una lista vacía para cada campo de la lista var
for i in var:
  globals()[str(i)] = []

In [ ]:
# Verificar que se tiene una lista vacía para cada campo
for i in vars:
  print(i, globals()[str(i)])

In [ ]:
#parameters

dd = pd.DataFrame()
ddt = pd.DataFrame()


#title Start actual driver
from selenium_profiles.webdriver import Chrome
from selenium_profiles.profiles import profiles
from selenium.webdriver.common.by import By  # locate elements
from selenium_profiles.utils.colab_utils import display, showscreen, show_html # virtual display
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from datetime import datetime

hora_inicio = datetime.now()

chromedriver_path = ChromeDriverManager(version="90.0.4430.24").install()

profile = profiles.Windows() # or .Android
profile["cdp"]["cores"] = None # Chrome 90 doesn't allow emulating cores :(driver = mydriver.start(profile, uc_driver=False, executable_path=chromedriver_path)

mydriver = Chrome(profile, executable_path=chromedriver_path)

display = display()
display.start_display()
driver = mydriver.start()

#Scrap law projects

for e in numbers:
  #clean
  estadoscrap = []
  autorlink = []
  fecha = []
  enlace = []
  for i in var:
    globals()[str(i)] = []


  print(e)
  hoy = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
  link = 'https://wb2server.congreso.gob.pe/spley-portal/#/expediente/2021/'+str(e)

  fecha.append(hoy)
  enlace.append(link)
  try:
    driver.get(link)  # test client hints
    time.sleep(2) # Tiempo de espera de 2s para ejecutar lo siguiente
    try:
      try:
        driver.find_element(By.XPATH,'//*[@id="p-fieldset-0-content"]/div/div[1]/div[9]/ul/li[3]/a[2]').click()
      except:
        driver.find_element(By.XPATH,'//*[@id="p-fieldset-1-content"]/div/div[1]/div[9]/ul/li[3]/a[2]').click()  #exp 2412
    except NoSuchElementException:
      print('not button')

    try:
      try:
        autorlin = driver.find_element(By.XPATH,'//*[@id="p-fieldset-0-content"]/div/div[1]/div[8]/ul/li/a').get_attribute('href')
      except StaleElementReferenceException:
        autorlin = ''

      status='no'
      autorlink.append(autorlin)

      for i,j in zip(var,path):
        #fill data into list
        try:
          globals()[str(i)].append(driver.find_element(By.XPATH,j).text)
        except NoSuchElementException:
          globals()[str(i)].append('')
      status = 'yes'
      estadoscrap.append(status)
      print('ok')
    except NoSuchElementException:
      pass
    try:
      table_element = driver.find_element(By.XPATH,path14)
      table_html = table_element.get_attribute('innerHTML')
      table_soup = BeautifulSoup(table_html, 'html.parser')
      table = table_soup.table
      dft = pd.read_html(str(table))[0]
      dft['enlace'] = link
    except NoSuchElementException:
      dft = pd.DataFrame()
  except NoSuchElementException:
    for i,j in zip(var,path):
        globals()[str(i)].append('')
    estadoscrap.append(status)
    #showscreen(driver)

  #Close Driver
  # driver.quit()
  # display.stop_display()
  # time.sleep(2)
  #append data to one law project
  data_tuples = list(zip(titulo,
                        periodo,
                        legisla,
                        fechapresenta,
                        proponente,
                        sumilla,
                        observaciones,
                        autor,
                        coautor,
                        adherentes,
                        grupoparla,
                        comisiones,
                        ultimoestado,
                        estadoscrap,
                        autorlink,
                        fecha,
                        enlace))
  #Convert data tuple into dataframe
  df = pd.DataFrame(data_tuples, columns=vars) # creates dataframe of each tuple in list
    #Append to main data
  dd = pd.concat([dd,df], axis=0, ignore_index=True) # Es la hoja "datoscongreso"
  ddt = pd.concat([ddt,dft], axis=0, ignore_index=True) # Es la hoja "datoscongresoseguimiento"

print(datetime.now() - hora_inicio)

#Close Driver
driver.quit()
display.stop_display()

In [ ]:
ddt['enlace'].value_counts()

In [ ]:
print('dd', dd.shape)
print('ddt', ddt.shape)

In [ ]:
dd

In [ ]:
dd['periodo'] = dd['periodo'].str.split('\n',expand=True)[1]
dd['comisiones'] = dd['comisiones'].str.split('\n',expand=True)[1]
dd['legisla'] = dd['legisla'].str.split('\n',expand=True)[1]
dd['fechapresenta'] = dd['fechapresenta'].str.split('\n',expand=True)[1]
dd['observaciones'] = dd['observaciones'].str.split('\n',expand=True)[1]
dd['autor'] = dd['autor'].str.split('\n',expand=True)[1]
dd['grupoparla'] = dd['grupoparla'].str.split('\n',expand=True)[1]
dd['ultimoestado'] = dd['ultimoestado'].str.split('\n',expand=True)[1]

# 06 Limpieza

## 06 01 Limpieza datos congreso

In [ ]:
has_line_break = dd['coautor'].str.contains('\n')
dd[has_line_break]

In [ ]:
dd['proponente'] = dd['proponente'].str.split('\n', expand=True)[1]
dd['coautor'] = dd['coautor'].str.split('\n', expand=True)[1]
dd['sumilla'] = dd['sumilla'].str.split('\n', expand=True)[1]
dd['adherentes'] = dd['adherentes'].str.split('\n', expand=True)[1]
dd.head(4)

In [ ]:
from pkg_resources import DEVELOP_DIST
dd['autor'] = dd['autor'].str.upper()
dd['autor'] = dd['autor'].str.replace(',','')

dd['autor']

## 06 02 Limpieza congreso seguimiento

In [ ]:
ddt.head()

In [ ]:
ddt.replace(np.nan, '', inplace = True)

In [ ]:
ddt['FECHA'] = [ x[5:] for x in ddt['FECHA']]
ddt['ESTADO PROCESAL'] = [ x[6:] for x in ddt['ESTADO PROCESAL']]
ddt['COMISIÓN'] = [ x[8:] for x in ddt['COMISIÓN']]
ddt['DETALLE'] = [ x[7:] for x in ddt['DETALLE']]
ddt.head()

In [ ]:
ddt['FECHA'] = pd.to_datetime(ddt['FECHA'], format='%d/%m/%Y')

ddt = ddt.sort_values(['enlace', 'FECHA'],
              ascending = [True, True])

ddt['contador'] = ddt.groupby(['enlace']).cumcount() + 1

#07 Creación nuevas variables

In [ ]:
ddt['min']= ddt.groupby(['enlace'])['FECHA'].transform(min)
ddt['max']= ddt.groupby(['enlace'])['FECHA'].transform(max)

# Obtención de la fecha máxima y mínima

ddt['min'] = pd.to_datetime(ddt['min'])
ddt['max'] = pd.to_datetime(ddt['max'])

# Cálculo del tiempo de duración

ddt['dias_proceso'] = (ddt['max'] - ddt['min']).dt.days # dt.days to remove the word "days"

# Cálculo del número de pasos del proyecto de ley

ddt['contador_total']= ddt.groupby(['enlace'])['contador'].transform(max)

In [ ]:
# Creación de la variable semanas_proceso:

ddt['semanas_proceso'] = ddt['dias_proceso']/140

# Creación de variable categórica para los días:

'''
De 0 a 20 semanas = 1
De 20 a 40 semanas = 2
y así sucesivamente
'''
# Redondear hacia arriba para obtener la variable categórica:

ddt['semanas_proceso_cat'] = ddt['semanas_proceso'].apply(np.ceil)

# Creación de variable categórica para el contador total:

ddt['contador_total_cat'] = (ddt['contador_total']/5).apply(np.ceil)

In [ ]:
# Creación de la variable brecha_fecha:

ddt['brecha_fecha'] = (datetime.now() - ddt['max']).dt.days

# Creación variable: contador^2

ddt['contador2'] = ddt['contador_total']*ddt['contador_total']

# Creación variable: dias_proceso^2

ddt['dias_proceso2'] = ddt['dias_proceso']*ddt['dias_proceso']

# 08 Merge

In [ ]:
df=pd.merge(dd, ddt[['contador_total','dias_proceso','enlace','semanas_proceso', 'semanas_proceso_cat', 'contador_total_cat', 'max']].drop_duplicates(subset='enlace', keep='last'), on="enlace", how="left", indicator=True)
df

In [ ]:
df = df.drop(['_merge'], axis=1)

In [ ]:
# Merge con datos congresistas

In [ ]:
list_1 = df['autor'].str.replace(',','').unique() #keys
list_2 = datoscongresistas['nombre'][(datoscongresistas['nombre']!='') & (datoscongresistas['fin'].str.contains('2026|2027|2028')==True)].unique()

bb = match_phrases_target(list_1, list_2, nn = 1)

In [ ]:
print('list_1',list_1.shape)
print('list_2',list_2.shape)
bb

In [ ]:
bb = bb[['list_1_phrases', 'list_2_ccpp']]
bb.columns = ['key_autor', 'key_congresista']
bb

In [ ]:
df1 = pd.merge(df, bb, left_on="autor", right_on="key_autor", how="left", indicator=True)
df1

In [ ]:
df1 = df1.drop(['_merge'], axis=1)

In [ ]:
df2 = pd.merge(df1, datoscongresistas[['nombre','votacion','inicio','fin','grupopartido','bancada','representa','condicion']][(datoscongresistas['nombre']!='') & (datoscongresistas['fin'].str.contains('2026|2027|2028')==True)],
               left_on="key_congresista", right_on="nombre", how="left", indicator=True)

In [ ]:
# exportación para ver resultados
lin = 'https://docs.google.com/spreadsheets/d/16vsPetGMBsRQfz-jIYL7JL32euMCYA6O8-qA48L4W8U/edit#gid=0'

In [ ]:
sh2 = gc.open_by_url(lin)
wks1 = sh2[3]
wks1.clear(start='A1', end=None)
wks1.set_dataframe(df2, 'A1')

In [ ]:
# Término del script